A ReadMe for this code can be found here: https://docs.google.com/document/d/1vt_oRtcYe5YmldGDwGQkv0YDgRYCZzbSVskOolH365g/edit?usp=sharing

In [ ]:
#@title Interpolate Sondes & Download

# Install necessary packages
#!pip install ipywidgets bs4

# Imports
import numpy as np
import pandas as pd
import requests
import io
import zipfile
import os
from google.colab import files
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display

# Constants
R_d = 287.05
epsilon = 0.622
base_url = "https://cw3e-datashare.ucsd.edu/CW3E_Radiosondes/"

# Fetch all available directories
def fetch_directories(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return [a['href'].strip('/') for a in soup.find_all('a', href=True)
            if a['href'].endswith('/') and not a['href'].startswith('?')]

# Fetch .txt files from "Text_Files" subdirectory
def fetch_files(directory):
    url = f"{base_url}{directory}/Text_Files/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return [f"Text_Files/{a['href']}" for a in soup.find_all('a', href=True)
            if a['href'].endswith('.txt')]

# Process a single file
def process_radiosonde_data(url):
    response = requests.get(url)
    data_str = response.text

    columns = [
        "Year", "Month", "Day", "Hour", "Minute", "Second",
        "Pressure (hPa)", "Temperature (C)", "Dew Point (C)",
        "Wind Direction (degrees)", "Wind Speed (m/s)", "Height (m)",
        "Latitude", "Longitude"
    ]

    data = [list(map(float, line.split())) for line in data_str.splitlines()
            if len(line.split()) == len(columns)]

    df = pd.DataFrame(data, columns=columns)
    df = df.sort_values(by="Pressure (hPa)").reset_index(drop=True)

    pressure_levels = np.arange(1000, 250 - 1, -25)
    df_interp = pd.DataFrame({"Pressure (hPa)": pressure_levels})

    for col in ["Temperature (C)", "Dew Point (C)", "Wind Direction (degrees)",
                "Wind Speed (m/s)", "Height (m)"]:
        df_interp[col] = np.interp(df_interp["Pressure (hPa)"], df["Pressure (hPa)"], df[col])

    df_interp[["Year", "Month", "Day", "Hour", "Minute", "Second", "Latitude", "Longitude"]] = \
        df.iloc[-1][["Year", "Month", "Day", "Hour", "Minute", "Second", "Latitude", "Longitude"]].values

    df_interp["Temperature (K)"] = df_interp["Temperature (C)"] + 273.15
    df_interp["Pressure (Pa)"] = df_interp["Pressure (hPa)"] * 100
    df_interp["Vapor Pressure (Pa)"] = 6.112 * np.exp(
        (17.67 * df_interp["Dew Point (C)"]) / (df_interp["Dew Point (C)"] + 243.5)) * 100
    df_interp["Specific Humidity"] = (
        epsilon * df_interp["Vapor Pressure (Pa)"]) / (
        df_interp["Pressure (Pa)"] - 0.378 * df_interp["Vapor Pressure (Pa)"])
    df_interp["Air Density (kg/m³)"] = df_interp["Pressure (Pa)"] / (R_d * df_interp["Temperature (K)"])
    df_interp["Water Vapor Flux (kg m-2 s-1)"] = df_interp["Air Density (kg/m³)"] * \
        df_interp["Specific Humidity"] * df_interp["Wind Speed (m/s)"]

    df_interp = df_interp[[
        "Year", "Month", "Day", "Hour", "Minute", "Second", "Pressure (hPa)",
        "Temperature (C)", "Dew Point (C)", "Wind Direction (degrees)",
        "Wind Speed (m/s)", "Height (m)", "Latitude", "Longitude",
        "Water Vapor Flux (kg m-2 s-1)"
    ]]

    filename = url.split("/")[-1].replace(".txt", "_intp.csv")
    df_interp.to_csv(filename, index=False)
    print(f"Processed {filename}")
    return filename

# GUI Setup
dir_dropdown = widgets.Dropdown(
    options=fetch_directories(base_url),
    description='Directory:',
    layout=widgets.Layout(width='50%')
)

file_select = widgets.SelectMultiple(
    options=[],
    description='Files:',
    layout=widgets.Layout(height='200px', width='50%')
)

process_button = widgets.Button(description='Process Selected Files', button_style='success')

# Update file list when directory changes
def update_file_list(change):
    if change['new']:
        selected_dir = change['new']
        file_paths = fetch_files(selected_dir)
        file_select.options = [f.split('/')[-1] for f in file_paths]

dir_dropdown.observe(update_file_list, names='value')

# Process files and trigger download
def process_and_download_files(b):
    selected_dir = dir_dropdown.value
    selected_filenames = file_select.value
    file_paths = fetch_files(selected_dir)

    selected_full_paths = [fp for fp in file_paths if fp.split('/')[-1] in selected_filenames]

    processed_files = []
    for fp in selected_full_paths:
        url = f"{base_url}{selected_dir}/{fp}"
        processed_files.append(process_radiosonde_data(url))

    # Zip and download
    zip_filename = "processed_files.zip"
    with zipfile.ZipFile(zip_filename, "w") as zipf:
        for file in processed_files:
            zipf.write(file)
            os.remove(file)

    files.download(zip_filename)
    print(f"\nAll files zipped and ready for download: {zip_filename}")

process_button.on_click(process_and_download_files)

# Display widgets
display(dir_dropdown, file_select, process_button)

In [ ]:
#@title Upload & Plot Sondes as Meteogram

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import BoundaryNorm, ListedColormap
from google.colab import files

# User input for NAME in title
name_input = input("Enter the NAME to be used in the title: ")

# Ask user for figure size with example sizes
print("Enter figure size in inches (width x height).")
print("Example sizes: 10x6 (standard), 12x8 (large), 8x5 (small).")

fig_width = float(input("Enter figure width (in inches): "))
fig_height = float(input("Enter figure height (in inches): "))

# Upload multiple CSV files
print("Please upload your CSV files (one per time step).")
uploaded = files.upload()

# Load and concatenate all uploaded CSV files
dfs = []
for file_name in uploaded.keys():
    df = pd.read_csv(file_name)
    dfs.append(df)
data = pd.concat(dfs, ignore_index=True)

# Round to nearest synoptic time
def round_to_synoptic_time(row):
    total_minutes = row["Hour"] * 60 + row["Minute"]
    rounded_hour = int(np.round(total_minutes / 60))  # Round to nearest hour
    row["Hour"] = rounded_hour % 24  # Ensure valid hour (0-23)
    row["Minute"] = 0
    row["Second"] = 0
    return row

data = data.apply(round_to_synoptic_time, axis=1)

data["Datetime"] = pd.to_datetime(data[["Year", "Month", "Day", "Hour", "Minute", "Second"]])

# Convert wind speed to knots
data["Wind Speed (knots)"] = data["Wind Speed (m/s)"] * 1.94384

# Sort data by time and pressure
data = data.sort_values(by=["Datetime", "Pressure (hPa)"], ascending=[True, False])

# Define color map for water vapor flux
levels = np.array([0.0, 0.025, 0.050, 0.075, 0.100, 0.125, 0.150, 0.175, 0.200, 0.225, 0.250, 0.300])
hex_colors = ["#FFFFFF", "#ffef29", "#ffdd27", "#ffd129", "#ffc123", "#fdb222", "#ff8f20", "#fe771f", "#ff5c1c", "#ff441c", "#ff2e1e", "#fe1818"]
cmap = ListedColormap(hex_colors)
norm = BoundaryNorm(levels, ncolors=len(hex_colors), extend="max")

# Interpolate water vapor flux hourly
data.set_index("Datetime", inplace=True)
data_flux = data.groupby("Pressure (hPa)")["Water Vapor Flux (kg m-2 s-1)"].apply(lambda group: group.resample("1h").interpolate()).reset_index()

data = data.reset_index().merge(data_flux, on=["Datetime", "Pressure (hPa)"], suffixes=("", "_interp"))

# Prepare mesh for shading
unique_times = data_flux["Datetime"].unique()

# Add buffer time steps to x-axis limits
buffer_time = np.timedelta64(1, 'h')
unique_times = np.insert(unique_times, 0, unique_times[0] - buffer_time)
unique_times = np.append(unique_times, unique_times[-1] + buffer_time)

pressures = np.sort(data["Pressure (hPa)"].unique())[::-1]
water_vapor_flux = np.zeros((len(pressures), len(unique_times)))

for i, time in enumerate(unique_times):
    group = data_flux[data_flux["Datetime"] == time]
    for j, pressure in enumerate(pressures):
        row = group[group["Pressure (hPa)"] == pressure]
        if not row.empty:
            water_vapor_flux[j, i] = row["Water Vapor Flux (kg m-2 s-1)"].values[0]

x, y = np.meshgrid(unique_times, pressures)

# Create the figure with user-defined size
fig, ax = plt.subplots(figsize=(fig_width, fig_height))

# Plot shaded water vapor flux
c = ax.contourf(x, y, water_vapor_flux, levels=levels, cmap=cmap, norm=norm, extend="max")

# Add contour lines with light grey borders
contours = ax.contour(x, y, water_vapor_flux, levels=levels, colors='black', linewidths=0.1)

# Plot wind barbs with a higher z-order to ensure they are on top of the contours
data_valid_winds = data.drop_duplicates(subset=["Datetime", "Pressure (hPa)"])
for time in data_valid_winds["Datetime"].unique():
    if time in unique_times:
        group = data_valid_winds[data_valid_winds["Datetime"] == time]
        pressures = group["Pressure (hPa)"]
        wind_speeds_knots = group["Wind Speed (knots)"]
        wind_dirs = group["Wind Direction (degrees)"]
        u = wind_speeds_knots * np.cos(np.radians(270 - wind_dirs))
        v = wind_speeds_knots * np.sin(np.radians(270 - wind_dirs))
        ax.barbs([np.datetime64(time)] * len(pressures), pressures, u, v, length=6, linewidth=0.4, zorder=10)

# Adjust color bar properties (shrink width & adjust height)
cbar = fig.colorbar(c, ax=ax, orientation="horizontal", label="Water Vapor Flux (kg m⁻² s⁻¹)",
                    pad=0.12, aspect=50, shrink=1.0)  # Adjust shrink and aspect for a thinner bar
cbar.ax.tick_params(labelsize=8)  # Reduce tick label font size
cbar.set_ticks(levels[::1])  # Show fewer tick labels
cbar.set_ticklabels([f"{l:.3f}" for l in levels[::1]])  # Format tick labels

# Customize X-axis: Tick marks every hour, labels only at (00, 03, 06, 09, 12, 15, 18, 21)
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))  # Set tick marks at every hour
ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))  # Ensure minor ticks exist at every hour

selected_hours = [0, 3, 6, 9, 12, 15, 18, 21]  # Only label these hours
tick_labels = [dt for dt in unique_times if pd.to_datetime(dt).hour in selected_hours]

ax.set_xticks(unique_times)  # Show tick marks for all hours
ax.set_xticklabels(["" if pd.to_datetime(dt).hour not in selected_hours else pd.to_datetime(dt).strftime("%HZ/%d") for dt in unique_times],
                   rotation=45, ha='right')

# Adjust other plot settings
ax.set_ylim(1050, 200)
ax.set_ylabel("Pressure (hPa)")
ax.set_title(f"CW3E {name_input} Radiosonde Observed Vapor Flux (kg m⁻² s⁻¹), Winds (knots)", loc='left')

plt.tight_layout()
plt.show()